<a href="https://colab.research.google.com/github/emello23/Alura-Gemini/blob/main/Agente_nova_saida.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import time
from IPython.display import display, Markdown
from tabulate import tabulate
from urllib.parse import quote_plus
import google.generativeai as genai # SDK do Gemini
import json # Para parsear a resposta do Gemini
from google.colab import userdata # Para acessar a API key
from datetime import datetime, timedelta # Importar datetime e timedelta

# --- Configuração do Agente e API Key ---
GOOGLE_API_KEY = None
gemini_model_instance = None # Renomeado para clareza

try:
    # O nome padrão para secrets é sem o sufixo _, a menos que você tenha nomeado especificamente assim.
    # Vou usar 'GOOGLE_API_KEY' como padrão, conforme a prática comum.
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    if not GOOGLE_API_KEY:
        display(Markdown("<font color='red'>**Erro Crítico: GOOGLE_API_KEY não encontrada nos Secrets do Colab.** "
                         "Por favor, configure-a com o nome 'GOOGLE_API_KEY'.</font>"))
    else:
        genai.configure(api_key=GOOGLE_API_KEY)
        gemini_model_instance = genai.GenerativeModel(
            model_name='gemini-1.5-flash-latest', # Modelo eficiente e capaz
            # A opção response_mime_type pode ser usada com modelos mais recentes
            # e configurações de segurança apropriadas.
            # generation_config=genai.types.GenerationConfig(
            #     response_mime_type="application/json"
            # )
        )
        display(Markdown("✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*"))
except Exception as e:
    display(Markdown(f"<font color='red'>**Erro Crítico ao configurar o SDK do Gemini:** {e}. "
                     "Verifique sua API Key, permissões e se o nome do secret está correto ('GOOGLE_API_KEY'). "
                     "O agente não funcionará sem isso.</font>"))
    # O script poderia parar aqui ou continuar com funcionalidades limitadas se houvesse um fallback.
    # Para este agente, o Gemini é essencial.

# --- Funções Utilitárias ---

def carregar_dados_viagem(caminho_arquivo: str) -> pd.DataFrame | None:
    """
    Carrega os dados de viagem do arquivo CSV e valida as colunas necessárias.
    """
    colunas_requeridas = ['cidade', 'pais', 'data_chegada', 'data_partida', 'hospedagem']
    try:
        df = pd.read_csv(caminho_arquivo)

        # Validar se as colunas requeridas existem
        if not all(col in df.columns for col in colunas_requeridas):
            colunas_faltando = [col for col in colunas_requeridas if col not in df.columns]
            display(Markdown(f"<font color='red'>**Erro: O arquivo `{caminho_arquivo}` não contém as colunas requeridas.** "
                             f"Faltam as seguintes colunas: {', '.join(colunas_faltando)}.</font>"))
            return None

        display(Markdown(f"### Arquivo de Viagem (`{caminho_arquivo}`)"))
        display(df[colunas_requeridas]) # Exibe apenas as colunas relevantes
        return df
    except FileNotFoundError:
        display(Markdown(f"<font color='red'>**Erro: Arquivo `{caminho_arquivo}` não encontrado.** Verifique o nome e o local.</font>"))
        return None
    except Exception as e:
        display(Markdown(f"<font color='red'>**Erro ao carregar o arquivo CSV `{caminho_arquivo}`:** {e}</font>"))
        return None

def gerar_links_pesquisa_google(cidade: str, pais: str) -> tuple[str, str, str]:
    """Gera links de pesquisa úteis para Google Search e Google Maps."""
    query_atracoes = f"principais atrações turísticas em {cidade} {pais}"
    link_google_search = f"https://www.google.com/search?q={quote_plus(query_atracoes)}"
    query_maps_atracoes = f"atrações turísticas em {cidade}, {pais}"
    link_google_maps_atracoes = f"https://www.google.com/maps/search/{quote_plus(query_maps_atracoes)}"
    query_maps_cidade = f"{cidade}, {pais}"
    link_google_maps_cidade = f"https://www.google.com/maps/place/{quote_plus(query_maps_cidade)}"
    return link_google_search, link_google_maps_atracoes, link_google_maps_cidade

# --- Definição do Agente Pesquisador de Atrações ---

class AgentePesquisadorAtracoes:
    """
    Um agente responsável por pesquisar atrações turísticas usando o Gemini
    e fornecer links úteis para pesquisa manual.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def _obter_sugestoes_gemini(self, cidade: str, pais: str, top_n: int = 15) -> list[dict]:
        """
        Ferramenta interna do agente para consultar o Gemini sobre atrações.
        """
        display(Markdown(f"🧠 *Agente consultando Gemini sobre atrações em: **{cidade}, {pais}** (Top {top_n})...*"))

        prompt = f"""
        Você é um assistente de planejamento de viagens altamente especializado e eficiente.
        Sua tarefa é identificar as {top_n} principais e mais recomendadas atrações turísticas para a cidade de {cidade}, localizada em {pais}.

        Critérios para sua seleção:
        1.  **Popularidade e Reconhecimento:** Atrações bem conhecidas e frequentemente visitadas.
        2.  **Qualidade das Avaliações:** Lugares geralmente com avaliações positivas (ex: acima de 4.0/5 estrelas).
        3.  **Relevância Cultural/Histórica:** Locais com significado importante.
        4.  **Diversidade de Experiências:** Inclua uma mistura de tipos, como museus, monumentos, parques, mercados, igrejas/catedrais, mirantes, etc., quando apropriado para a cidade.
        5.  **Singularidade:** Atrações que oferecem uma experiência única ou icônica da cidade/região.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando uma atração,
        contendo EXATAMENTE as seguintes chaves:
        - "nome": (string) O nome oficial e completo da atração.
        - "tipo_principal": (string) A categoria principal da atração (ex: "Museu de Arte", "Catedral Gótica", "Parque Urbano", "Mercado Histórico", "Monumento Nacional", "Mirante Panorâmico"). Seja específico.
        - "descricao_enxuta": (string) Uma descrição concisa e informativa (1-2 frases) que justifique sua inclusão, destacando seus principais atrativos.
        - "avaliacao_geral": (string, opcional) Uma estimativa da avaliação geral se for amplamente conhecida (ex: "Excelente", "Muito Popular", "4.5/5 estrelas"). Use "N/A" se não houver uma estimativa clara.
        - "destaque_principal": (string) O motivo principal pelo qual um turista deveria visitar (ex: "Vistas incríveis da cidade", "Coleção de arte renascentista", "Arquitetura impressionante", "Atmosfera vibrante").

        Exemplo de um item na lista JSON:
        {{
          "nome": "Museu do Louvre",
          "tipo_principal": "Museu de Arte e Antiguidades",
          "descricao_enxuta": "Um dos maiores e mais visitados museus do mundo, lar de obras-primas como a Mona Lisa e a Vênus de Milo.",
          "avaliacao_geral": "4.7/5 estrelas",
          "destaque_principal": "Coleção de arte de renome mundial"
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.4, # Um pouco mais factual, menos aleatório
                    max_output_tokens=3072 # Espaço suficiente para a lista JSON
                ),
                safety_settings=self.safety_settings
            )

            json_text = response.text.strip()
            # Tentativa robusta de extrair JSON, mesmo que o modelo adicione ```json ... ```
            if json_text.startswith("```json"):
                json_text = json_text[7:]
            if json_text.endswith("```"):
                json_text = json_text[:-3]
            json_text = json_text.strip()

            if not json_text:
                display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia para {cidade}.</font>"))
                return []

            atracoes = json.loads(json_text)
            # Garantir que é uma lista, mesmo que o Gemini retorne algo diferente
            if not isinstance(atracoes, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para {cidade}.</font>"))
                 return []

            display(Markdown(f"✅ *Gemini retornou {len(atracoes)} sugestões de atrações para {cidade}.*"))
            return atracoes[:top_n]
        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return []
        except Exception as e:
            # Captura de erros mais genéricos, como problemas de API (quota, etc.)
            # ou bloqueios de segurança não esperados.
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return []

    def pesquisar_destino(self, cidade: str, pais: str, top_n_sugestoes: int = 15) -> dict:
        """
        Executa a pesquisa de atrações para um destino específico.
        """
        display(Markdown(f"\n### 🌍 Agente Iniciando Pesquisa para: **{cidade}, {pais}**"))

        links = gerar_links_pesquisa_google(cidade, pais)
        sugestoes_gemini = self._obter_sugestoes_gemini(cidade, pais, top_n=top_n_sugestoes)

        display(Markdown(f"--- Pesquisa para **{cidade}, {pais}** finalizada pelo agente. ---"))
        return {
            "cidade": cidade,
            "pais": pais,
            "link_google_search": links[0],
            "link_google_maps_atracoes": links[1],
            "link_google_maps_cidade": links[2],
            "sugestoes_gemini": sugestoes_gemini
        }

# --- Definição do Agente Montador de Roteiros ---

class MontadorDeRoteiros:
    """
    Um agente responsável por organizar as atrações por dia, considerando proximidade,
    hospedagem como ponto de partida/chegada, e adicionando elementos para tornar o roteiro prazeroso e divertido.
    """
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [ # Configurações de segurança para o Gemini
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def montar_roteiro(self, cidade: str, pais: str, atracoes: list, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        """
        Monta um roteiro diário otimizado para as atrações, considerando a hospedagem e datas.
        """
        from datetime import datetime
        delta = datetime.strptime(data_partida, "%Y-%m-%d") - datetime.strptime(data_chegada, "%Y-%m-%d")
        dias_viagem = delta.days + 1 # Inclui o dia de chegada e partida

        atracoes_str = json.dumps(atracoes, ensure_ascii=False, indent=2)

        prompt = f"""
        Você é um especialista em planejamento de roteiros de viagem, focado em otimização e experiência do viajante.
        Sua tarefa é criar um roteiro diário detalhado para uma viagem a {cidade}, {pais}, com base nas atrações fornecidas.
        A viagem será de {data_chegada} a {data_partida}, totalizando {dias_viagem} dias.
        A hospedagem principal é em: {hospedagem}.

        Atrações disponíveis (formato JSON):
{atracoes_str}

        Critérios para o Roteiro:
        1.  **Otimização Geográfica:** Agrupe atrações próximas no mesmo dia para minimizar deslocamentos.
        2.  **Fluxo Lógico:** Organize as visitas de forma sequencial e eficiente.
        3.  **Ritmo Agradável:** Considere tempos de deslocamento, duração estimada das visitas e pausas (almoço, descanso). Evite sobrecarregar os dias.
        4.  **Hospedagem como Base:** O roteiro deve idealmente começar e terminar o dia próximo à área da hospedagem, se possível.
        5.  **Flexibilidade:** Mencione que o roteiro é uma sugestão e pode ser adaptado.
        6.  **Sugestões Adicionais:** Inclua sugestões gerais de onde comer na região das atrações do dia, ou atividades noturnas, se apropriado para a cidade.
        7.  **Formato Diário:** Organize a saída por dia da viagem.

        Formato da Resposta:
        Retorne **estritamente uma lista JSON**. Cada elemento da lista deve ser um dicionário representando um dia do roteiro,
        contendo EXATAMENTE as seguintes chaves:
        - "dia": (integer) O número do dia da viagem (começando em 1).
        - "data": (string) A data correspondente ao dia do roteiro no formato YYYY-MM-DD.
        - "atividades_sugeridas": (lista de strings) Uma lista ordenada das atrações e atividades sugeridas para este dia, com descrições concisas (ex: "Visitar o Museu do Louvre (reservar 3-4 horas)", "Passeio pelo Jardim de Tuileries (1-2 horas)", "Almoço na região do museu"). Inclua tempos estimados ou sugestões de duração quando possível.
        - "observacoes": (string) Sugestões adicionais para o dia, como opções de transporte, áreas para almoço/jantar, ou dicas específicas para as atrações do dia. Use "N/A" se não houver observações relevantes.

        Exemplo de um item na lista JSON:
        {{
          "dia": 1,
          "data": "{data_chegada}",
          "atividades_sugeridas": [
            "Chegada e check-in na hospedagem ({hospedagem})",
            "Passeio exploratório pela vizinhança da hospedagem",
            "Jantar em um restaurante local"
          ],
          "observacoes": "Aproveite para se ambientar na área."
        }}

        Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
        """

        display(Markdown(f"🧠 *Agente montando roteiro para: **{cidade}, {pais}** ({dias_viagem} dias)...*"))

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.7, # Um pouco mais criativo para o roteiro
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_text.startswith("```json"):
                roteiro_text = roteiro_text[7:]
            if roteiro_text.endswith("```"):
                roteiro_text = roteiro_text[:-3]
            roteiro_text = roteiro_text.strip()


            if not roteiro_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar montar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}


            roteiro = json.loads(roteiro_text)
            if not isinstance(roteiro, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini montou um roteiro com {len(roteiro)} dias para {cidade}.*"))
            return {"roteiro": roteiro}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para montar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}


class AgenteOtimizadorRoteiros:
    def __init__(self, modelo_llm: genai.GenerativeModel):
        if modelo_llm is None:
            raise ValueError("O modelo LLM (Gemini) não pode ser None para este agente.")
        self.modelo = modelo_llm
        self.safety_settings = [
            {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
            {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
        ]

    def otimizar_roteiro(self, roteiro_original: list, cidade: str, pais: str, hospedagem: str, data_chegada: str, data_partida: str) -> dict:
        display(Markdown(f"🧠 *Agente Otimizador revisando roteiro para: **{cidade}, {pais}**...*"))
        roteiro_str = json.dumps(roteiro_original, ensure_ascii=False, indent=2)

        prompt = f"""
Você é um Agente Otimizador de Roteiros de Viagem, especialista em logística e experiência do viajante.
Sua missão é revisar e otimizar um roteiro de viagem existente para {cidade}, {pais}, com base nos seguintes critérios:

Roteiro Original (JSON):
{roteiro_str}

Critérios de Otimização:
1.  **Melhoria do Fluxo:** Reorganize as atividades dentro de cada dia ou entre dias para criar um fluxo mais lógico e eficiente, minimizando o tempo de deslocamento.
2.  **Balanceamento:** Distribua as atividades de forma mais equilibrada ao longo dos dias, evitando dias excessivamente cheios ou vazios.
3.  **Considerar Horários:** Leve em conta horários de funcionamento de atrações, horários de pico e o melhor momento do dia para visitar certos locais (ex: pôr do sol em um mirante).
4.  **Sugestões de Transição:** Adicione sugestões sobre como se deslocar entre as atrações (transporte público, caminhada, táxi) e tempos estimados.
5.  **Refinar Sugestões Adicionais:** Melhore as sugestões de alimentação e atividades noturnas com base no roteiro otimizado.
6.  **Formato de Saída:** Mantenha o mesmo formato JSON da lista de dias, mas com as atividades e observações revisadas.

Formato da Resposta:
Retorne **estritamente uma lista JSON**, seguindo o mesmo formato descrito para o roteiro original,
mas com as otimizações aplicadas.

Exemplo de um item otimizado na lista JSON:
{{
  "dia": 1,
  "data": "{data_chegada}",
  "atividades_sugeridas": [
    "Chegada no aeroporto, traslado para a hospedagem ({hospedagem})",
    "Check-in e breve descanso",
    "Passeio a pé pela área histórica próxima à hospedagem (1-2 horas)",
    "Jantar em um restaurante recomendado na vizinhança"
  ],
  "observacoes": "Considere usar transporte público ou táxi do aeroporto. Explore a pé a área ao redor do hotel para se familiarizar com o local."
}}

Garanta que a saída seja SOMENTE a lista JSON, sem nenhum texto introdutório, comentários ou formatação adicional.
"""

        try:
            response = self.modelo.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.8, # Um pouco mais criativo para otimizar
                    max_output_tokens=4096 # Espaço suficiente para o roteiro completo
                ),
                safety_settings=self.safety_settings
            )

            roteiro_otimizado_text = response.text.strip()
            # Tentativa robusta de extrair JSON
            if roteiro_otimizado_text.startswith("```json"):
                roteiro_otimizado_text = roteiro_otimizado_text[7:]
            if roteiro_otimizado_text.endswith("```"):
                roteiro_otimizado_text = roteiro_otimizado_text[:-3]
            roteiro_otimizado_text = roteiro_otimizado_text.strip()

            if not roteiro_otimizado_text:
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou uma resposta vazia ao tentar otimizar o roteiro para {cidade}.</font>"))
                 return {"erro": "resposta vazia do Gemini"}

            roteiro_otimizado = json.loads(roteiro_otimizado_text)
            if not isinstance(roteiro_otimizado, list):
                 display(Markdown(f"<font color='orange'>Aviso: Gemini retornou um formato inesperado (não uma lista) para o roteiro otimizado de {cidade}.</font>"))
                 return {"erro": "formato inesperado do Gemini"}

            display(Markdown(f"✅ *Gemini otimizou o roteiro com {len(roteiro_otimizado)} dias para {cidade}.*"))
            return {"roteiro_otimizado": roteiro_otimizado}

        except json.JSONDecodeError as e:
            display(Markdown(f"<font color='red'>**Erro (JSONDecodeError) ao processar resposta do Gemini para o roteiro otimizado de {cidade}:** {e}. "
                             "Isso geralmente ocorre se o modelo não retornar um JSON válido.</font>"))
            if hasattr(response, 'text'):
                display(Markdown(f"<pre>Resposta Bruta do Gemini:\n{response.text}</pre>"))
            else:
                 display(Markdown(f"<pre>Nenhuma resposta de texto recebida do Gemini.</pre>"))
            return {"erro": f"JSONDecodeError: {e}"}
        except Exception as e:
            display(Markdown(f"<font color='red'>**Erro inesperado ao consultar Gemini para otimizar o roteiro de {cidade}:** {e}</font>"))
            if hasattr(response, 'prompt_feedback'):
                display(Markdown(f"Feedback do Prompt: {response.prompt_feedback}"))
            return {"erro": f"Erro inesperado: {e}"}
# ----------
def executar_planejador_viagem(caminho_arquivo_csv: str) -> list:
    """
    Orquestra o processo de planejamento de viagem: carrega dados, pesquisa atrações
    e monta roteiros para cada destino no arquivo CSV.
    """
    display(Markdown("## ✈️ Iniciando Planejamento de Viagem..."))

    # 1. Carregar os dados de viagem
    df_viagens = carregar_dados_viagem(caminho_arquivo_csv)
    if df_viagens is None or df_viagens.empty:
        display(Markdown("<font color='red'>**Erro: Não foi possível carregar os dados de viagem ou o arquivo está vazio.**</font>"))
        return []

    # Garantir que o modelo Gemini esteja configurado
    if gemini_model_instance is None:
         display(Markdown("<font color='red'>**Erro: O modelo Gemini não foi configurado corretamente. O planejamento não pode continuar.**</font>"))
         return []

    agente_pesquisador = AgentePesquisadorAtracoes(gemini_model_instance)
    agente_roteirista = MontadorDeRoteiros(gemini_model_instance)
    agente_otimizador = AgenteOtimizadorRoteiros(gemini_model_instance)


    resultados_viagem = []

    # Iterar sobre cada linha do DataFrame (cada destino)
    for index, linha in df_viagens.iterrows():
        cidade = linha['cidade']
        pais = linha['pais']
        data_chegada = linha['data_chegada']
        data_partida = linha['data_partida']
        hospedagem = linha['hospedagem']

        display(Markdown(f"\n--- Processando Destino: **{cidade}, {pais}** ---"))

        # Validar datas
        try:
            datetime.strptime(data_chegada, "%Y-%m-%d")
            datetime.strptime(data_partida, "%Y-%m-%d")
            if datetime.strptime(data_chegada, "%Y-%m-%d") > datetime.strptime(data_partida, "%Y-%m-%d"):
                 display(Markdown(f"<font color='orange'>Aviso: Data de chegada ({data_chegada}) é posterior à data de partida ({data_partida}) para {cidade}. Pulando este destino.</font>"))
                 continue # Pula para o próximo destino
        except ValueError:
            display(Markdown(f"<font color='orange'>Aviso: Formato de data inválido para {cidade}. Esperado YYYY-MM-DD. Pulando este destino.</font>"))
            continue # Pula para o próximo destino


        # 2. Pesquisar atrações com o Agente Pesquisador
        resultado_pesquisa = agente_pesquisador.pesquisar_destino(cidade, pais)
        sugestoes_atracoes = resultado_pesquisa.get("sugestoes_gemini", [])

        if not sugestoes_atracoes:
            display(Markdown(f"<font color='orange'>Aviso: Nenhuma sugestão de atração encontrada para {cidade}. Não será possível montar um roteiro detalhado.</font>"))
            # Ainda armazena os links e info básica
            resultados_viagem.append({
                "cidade": cidade,
                "pais": pais,
                "data_chegada": data_chegada,
                "data_partida": data_partida,
                "hospedagem": hospedagem,
                "pesquisa": resultado_pesquisa,
                "roteiro": None, # Indica que o roteiro não pôde ser gerado
                "roteiro_otimizado": None # Indica que o roteiro otimizado não pôde ser gerado
            })
            continue # Pula para o próximo destino

        # 3. Montar o roteiro inicial com o Agente Montador
        resultado_roteiro_inicial = agente_roteirista.montar_roteiro(
            cidade, pais, sugestoes_atracoes, hospedagem, data_chegada, data_partida
        )

        roteiro_final = None
        if resultado_roteiro_inicial and resultado_roteiro_inicial.get("roteiro"):
            # 4. Otimizar o roteiro com o Agente Otimizador
            resultado_roteiro_otimizado = agente_otimizador.otimizar_roteiro(
                resultado_roteiro_inicial["roteiro"], cidade, pais, hospedagem, data_chegada, data_partida
            )
            roteiro_final = resultado_roteiro_otimizado.get("roteiro_otimizado")
            if not roteiro_final:
                 display(Markdown(f"<font color='orange'>Aviso: Não foi possível otimizar o roteiro para {cidade}. Usando o roteiro inicial (se disponível).</font>"))
                 roteiro_final = resultado_roteiro_inicial.get("roteiro") # Fallback para roteiro inicial


        # Armazenar os resultados para este destino
        resultados_viagem.append({
            "cidade": cidade,
            "pais": pais,
            "data_chegada": data_chegada,
            "data_partida": data_partida,
            "hospedagem": hospedagem,
            "pesquisa": resultado_pesquisa,
            "roteiro": resultado_roteiro_inicial.get("roteiro") if resultado_roteiro_inicial else None,
            "roteiro_otimizado": roteiro_final
        })

    display(Markdown("\n## ✅ Planejamento de Viagem Concluído!"))
    return resultados_viagem

✅ *SDK do Gemini e modelo (`gemini-1.5-flash-latest`) configurados com sucesso.*

In [2]:
dados_finais_viagem = dados_compilados = executar_planejador_viagem("/content/europa.csv")

## ✈️ Iniciando Planejamento de Viagem...

### Arquivo de Viagem (`/content/europa.csv`)

,cidade,pais,data_chegada,data_partida,hospedagem
0,Roma,Itália,2025-09-13,2025-09-16,Via Candia n.143 CAP: 00192
1,Florença,Itália,2025-09-16,2025-09-19,Hotel Bodoni
2,Viena,Aústria,2025-09-20,2025-09-24,Hotel-Pension Wild



--- Processando Destino: **Roma, Itália** ---


### 🌍 Agente Iniciando Pesquisa para: **Roma, Itália**

...

🧠 *Agente montando roteiro para: **Viena, Aústria** (5 dias)...*

✅ *Gemini montou um roteiro com 5 dias para Viena.*

🧠 *Agente Otimizador revisando roteiro para: **Viena, Aústria**...*

✅ *Gemini otimizou o roteiro com 5 dias para Viena.*


## ✅ Planejamento de Viagem Concluído!

In [3]:
import json

output_json_path = "/content/planejamento_viagem_completo.json"

if 'dados_finais_viagem' in locals() and dados_finais_viagem:
    try:
        with open(output_json_path, "w", encoding="utf-8") as f:
            json.dump(dados_finais_viagem, f, ensure_ascii=False, indent=4)
        display(Markdown(f"✅ **Planejamento completo da viagem salvo em:** `{output_json_path}`"))
    except Exception as e:
        display(Markdown(f"<font color='red'>**Erro ao salvar o planejamento em JSON:** {e}</font>"))
else:
    display(Markdown("⚠️ **Nenhum dado de planejamento de viagem encontrado para salvar em JSON.** Certifique-se de que o planejador foi executado."))


✅ **Planejamento completo da viagem salvo em:** `/content/planejamento_viagem_completo.json`

In [4]:
!pip install fpdf2 # Instala a biblioteca para gerar PDF

import json
from fpdf import FPDF # Biblioteca para gerar PDF

input_json_path = "/content/planejamento_viagem_completo.json"
output_pdf_path = "/content/planejamento_viagem_completo.pdf"

try:
    with open(input_json_path, "r", encoding="utf-8") as f:
        planejamento_completo = json.load(f)

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", "B", 16)
    pdf.cell(200, 10, txt="Planejamento Completo da Viagem", ln=True, align="C")
    pdf.ln(10)

    for destino_info in planejamento_completo:
        pdf.set_font("Arial", "B", 14)
        pdf.cell(200, 10, txt=f"Cidade: {destino_info.get('cidade', 'N/A')} ({destino_info.get('pais', 'N/A')})", ln=True)
        pdf.set_font("Arial", "", 12)
        pdf.multi_cell(0, 8, txt=f"Hospedagem: {destino_info.get('hospedagem', 'N/A')}")
        pdf.multi_cell(0, 8, txt=f"Período: {destino_info.get('data_chegada', 'N/A')} a {destino_info.get('data_partida', 'N/A')}")
        pdf.ln(5)

        if destino_info.get('roteiro_otimizado'):
            pdf.set_font("Arial", "B", 12)
            pdf.cell(200, 10, txt="Roteiro Otimizado:", ln=True)
            pdf.set_font("Arial", "", 10)
            for dia_roteiro in destino_info['roteiro_otimizado']:
                pdf.multi_cell(0, 6, txt=f"Dia {dia_roteiro.get('dia', 'N/A')}: {dia_roteiro.get('atividades', 'N/A')}")
                if dia_roteiro.get('restaurantes'):
                    pdf.set_font("Arial", "I", 10)
                    pdf.multi_cell(0, 6, txt="  Sugestões de Restaurantes:")
                    for restaurante in dia_roteiro['restaurantes']:
                        pdf.multi_cell(0, 6, txt=f"    - {restaurante.get('nome', 'N/A')} ({restaurante.get('faixa_preco', 'N/A')})")
                    pdf.set_font("Arial", "", 10)
                if dia_roteiro.get('detalhes_atracoes'):
                    pdf.set_font("Arial", "I", 10)
                    pdf.multi_cell(0, 6, txt="  Detalhes das Atrações:")
                    for atracao_detalhe in dia_roteiro['detalhes_atracoes']:
                        pdf.multi_cell(0, 6, txt=f"    - {atracao_detalhe.get('nome', 'N/A')} (Preço: {atracao_detalhe.get('preco_ingresso', 'N/A')}, Link: {atracao_detalhe.get('link_ingresso', 'N/A')})")
                    pdf.set_font("Arial", "", 10)
                pdf.ln(2)
        else:
            pdf.set_font("Arial", "", 10)
            pdf.multi_cell(0, 8, txt="Nenhum roteiro otimizado disponível para esta cidade.")
        pdf.ln(10)

    pdf.output(output_pdf_path)
    display(Markdown(f"✅ **Planejamento completo da viagem salvo em PDF:** `{output_pdf_path}`"))
except FileNotFoundError:
    display(Markdown(f"<font color='red'>**Erro: Arquivo JSON de planejamento não encontrado em `{input_json_path}`.**</font>"))
except json.JSONDecodeError as e:
    display(Markdown(f"<font color='red'>**Erro ao ler o arquivo JSON de planejamento:** {e}. Verifique se o arquivo está formatado corretamente.</font>"))
except Exception as e:
    display(Markdown(f"<font color='red'>**Erro inesperado ao gerar o PDF:** {e}</font>"))


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.7/245.7 kB 10.1 MB/s eta 0:00:00


/tmp/ipython-input-4-1380054526.py:15: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 16)
/tmp/ipython-input-4-1380054526.py:16: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Planejamento Completo da Viagem", ln=True, align="C")
/tmp/ipython-input-4-1380054526.py:16: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Planejamento Completo da Viagem", ln=True, align="C")
/tmp/ipython-input-4-1380054526.py:20: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 14)
/tmp/ipython-input-4-1380054526.py:21: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt=f"Cidade: {destino_info.

<font color='red'>**Erro inesperado ao gerar o PDF:** Not enough horizontal space to render a single character</font>

In [5]:
import json
# As funções 'display' e 'Markdown' são específicas de ambientes como Jupyter Notebook ou Google Colab.
# Se estiver usando um, este import é necessário.
from IPython.display import display, Markdown

# Verifica se a variável 'dados_finais_viagem' existe e não está vazia
if 'dados_finais_viagem' in locals() and dados_finais_viagem:

    # Salva o planejamento completo da viagem em um arquivo JSON
    output_filename = "planejamento_viagem_completo.json"
    with open(output_filename, "w", encoding="utf-8") as f:
        json.dump(dados_finais_viagem, f, ensure_ascii=False, indent=4)

    display(Markdown(f"✅ **Planejamento completo da viagem salvo em `{output_filename}`**"))

    # Mostra o roteiro otimizado para cada cidade
    for destino_info in dados_finais_viagem:
        cidade = destino_info["cidade"]
        pais = destino_info["pais"]
        roteiro_otimizado = destino_info.get("roteiro_otimizado")
        sugestoes_restaurantes = destino_info.get("sugestoes_restaurantes")
        atracoes_detalhadas = destino_info.get("atracoes_detalhadas")

        display(Markdown(f"### 🗺️ Roteiro Otimizado para: **{cidade}, {pais}**"))

        if roteiro_otimizado:
            for dia_roteiro in roteiro_otimizado:
                display(Markdown(f"#### Dia {dia_roteiro['dia']} ({dia_roteiro['data']}):"))

                display(Markdown("**Atividades:**"))
                for atividade in dia_roteiro["atividades_sugeridas"]:
                    display(Markdown(f"- {atividade}"))

                if dia_roteiro.get("observacoes") and dia_roteiro["observacoes"] != "N/A":
                    display(Markdown(f"*Observações:* {dia_roteiro['observacoes']}"))

                # Sugestões de Restaurantes para o dia
                if sugestoes_restaurantes and str(dia_roteiro["dia"]) in sugestoes_restaurantes:
                    display(Markdown(f"**Sugestões de Restaurantes para o Dia {dia_roteiro['dia']}:**"))
                    for categoria, restaurantes in sugestoes_restaurantes[str(dia_roteiro['dia'])].items():
                        display(Markdown(f"* {categoria}:"))
                        for restaurante in restaurantes:
                            display(Markdown(f"    - {restaurante['nome']} ({restaurante['faixa_preco']}) - Avaliação: {restaurante['avaliacao_geral']} - Link: [Google Maps]({restaurante['link_google_maps']})"))

                # Detalhes de Atrações para o dia
                if atracoes_detalhadas:
                    display(Markdown(f"**Detalhes de Atrações para o Dia {dia_roteiro['dia']}:**"))
                    for atracao in atracoes_detalhadas:
                        display(Markdown(f"- **{atracao['nome']}** (Preço: {atracao.get('preco_ingresso_euros', 'N/A')}€) - [Comprar Ingresso]({atracao.get('link_compra_ingresso', '#')})"))

                display(Markdown("---"))
        else:
            display(Markdown(f"<font color='orange'>Aviso: Roteiro otimizado não disponível para {cidade}.</font>"))
else:
    display(Markdown("### ⚠️ Nenhum dado de viagem final disponível para exibição. Execute o planejador primeiro."))

✅ **Planejamento completo da viagem salvo em `planejamento_viagem_completo.json`**

### 🗺️ Roteiro Otimizado para: **Roma, Itália**

#### Dia 1 (2025-09-13):

**Atividades:**

- Chegada no aeroporto Fiumicino (FCO), traslado para a hospedagem (Via Candia n.143 CAP: 00192) - Utilize o Leonardo Express (trem) até a estação Termini, depois táxi ou transporte público até o hotel (aprox. 1 hora e 30 minutos)

...

- Compras de lembranças no centro da cidade (aprox. 1-2 horas)

- Jantar de despedida em um restaurante de sua escolha

- Check-out do Hotel-Pension Wild

*Observações:* Combine o Prater com o Donauturm para aproveitar o pôr do sol.  Retorne ao centro para compras de lembranças.

---

In [6]:
import json

# Caminho para salvar o arquivo JSON
json_output_path = "/content/planejamento_viagem_completo.json"

# Verifica se 'dados_finais_viagem' está definido e não é None
if 'dados_finais_viagem' in locals() and dados_finais_viagem is not None:
    try:
        with open(json_output_path, 'w', encoding='utf-8') as f:
            json.dump(dados_finais_viagem, f, ensure_ascii=False, indent=4)
        print(f"✅ Planejamento completo da viagem salvo em {json_output_path}")
    except Exception as e:
        print(f"❌ Erro ao salvar o planejamento em JSON: {e}")
else:
    print("⚠️ Variável 'dados_finais_viagem' não encontrada ou está vazia. Certifique-se de que o planejador foi executado.")


✅ Planejamento completo da viagem salvo em /content/planejamento_viagem_completo.json


In [10]:
import json
from fpdf import FPDF
import os

json_input_path = "/content/planejamento_viagem_completo.json"
pdf_output_path = "/content/planejamento_viagem_completo.pdf"

if not os.path.exists(json_input_path):
    print(f"❌ Erro: Arquivo JSON de planejamento não encontrado em {json_input_path}.Certifique-se de que a célula anterior foi executada e o JSON foi salvo.")

else:
    try:
        with open(json_input_path, "r", encoding="utf-8") as f:
            planejamento_viagem = json.load(f)

        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", "B", 16)
        pdf.cell(200, 10, txt="Planejamento de Viagem Otimizado", ln=True, align="C")
        pdf.set_font("Arial", "", 12)
        pdf.ln(10)

        for destino_info in planejamento_viagem:
            pdf.set_font("Arial", "B", 14)
            pdf.cell(200, 10, txt=f"Cidade: {destino_info.get('cidade', 'N/A')} ({destino_info.get('pais', 'N/A')})", ln=True)
            pdf.set_font("Arial", "", 12)
            pdf.multi_cell(0, 8, txt=f"Hospedagem: {destino_info.get('hospedagem', 'N/A')}")
            pdf.multi_cell(0, 8, txt=f"Período: {destino_info.get('data_chegada', 'N/A')} a {destino_info.get('data_partida', 'N/A')}")
            pdf.ln(5)

            if "roteiro_otimizado" in destino_info and destino_info["roteiro_otimizado"]:
                pdf.set_font("Arial", "B", 12)
                pdf.cell(200, 10, txt="Roteiro Otimizado:", ln=True)
                pdf.set_font("Arial", "", 10)
                for dia_roteiro in destino_info["roteiro_otimizado"]:
                    pdf.multi_cell(0, 6, txt=f"Dia {dia_roteiro.get('dia', 'N/A')}: {dia_roteiro.get('atividades', 'N/A')}")
                    if "restaurantes" in dia_roteiro and dia_roteiro["restaurantes"]:
                        pdf.multi_cell(0, 6, txt="  Sugestões de Restaurantes:")
                        for cat, rest_list in dia_roteiro["restaurantes"] .items():
                            pdf.multi_cell(0, 6, txt=f"    {cat}:")
                            for rest in rest_list:
                                pdf.multi_cell(0, 6, txt=f"      - {rest.get('nome', 'N/A')} ({rest.get('faixa_preco', 'N/A')})")
                    if "atracoes_detalhadas" in dia_roteiro and dia_roteiro["atracoes_detalhadas"]:
                        pdf.multi_cell(0, 6, txt="  Detalhes das Atrações:")
                        for atracao_detalhe in dia_roteiro["atracoes_detalhadas"]:
                            pdf.multi_cell(0, 6, txt=f"    - {atracao_detalhe.get('nome', 'N/A')}: Preço: {atracao_detalhe.get('preco_ingresso', 'N/A')}, Link: {atracao_detalhe.get('link_ingresso', 'N/A')}")
            else:
                pdf.multi_cell(0, 8, txt="Nenhum roteiro otimizado disponível para esta cidade.")
            pdf.ln(10)

        pdf.output(pdf_output_path)
        print(f"✅ Planejamento completo da viagem salvo em PDF: {pdf_output_path}")
    except Exception as e:
        print(f"❌ Erro ao gerar o PDF: {e}")


❌ Erro ao gerar o PDF: Not enough horizontal space to render a single character


/tmp/ipython-input-10-3963205386.py:18: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "B", 16)
/tmp/ipython-input-10-3963205386.py:19: DeprecationWarning: The parameter "txt" has been renamed to "text" in 2.7.6
  pdf.cell(200, 10, txt="Planejamento de Viagem Otimizado", ln=True, align="C")
/tmp/ipython-input-10-3963205386.py:19: DeprecationWarning: The parameter "ln" is deprecated since v2.5.2. Instead of ln=True use new_x=XPos.LMARGIN, new_y=YPos.NEXT.
  pdf.cell(200, 10, txt="Planejamento de Viagem Otimizado", ln=True, align="C")
/tmp/ipython-input-10-3963205386.py:20: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be removed
  pdf.set_font("Arial", "", 12)
/tmp/ipython-input-10-3963205386.py:24: DeprecationWarning: Substituting font arial by core font helvetica - This is deprecated since v2.7.8, and will soon be

In [ ]:
import osimport jsonfrom fpdf import FPDF# Caminhos dos arquivosjson_input_path = "/content/planejamento_viagem_completo.json"pdf_output_path = "/content/planejamento_viagem_completo.pdf"# Verificar se o arquivo JSON existeif not os.path.exists(json_input_path):    print(f"❌ Erro: Arquivo JSON de planejamento não encontrado em {json_input_path}.")    print("Certifique-se de que a célula anterior foi executada e o JSON foi salvo.")else:    try:        # Carregar dados do JSON        with open(json_input_path, "r", encoding="utf-8") as f:            planejamento_viagem = json.load(f)        # Criar PDF        pdf = FPDF()        pdf.add_page()        pdf.set_font("Arial", "B", 16)        pdf.cell(200, 10, txt="Planejamento de Viagem Otimizado", ln=True, align="C")        pdf.set_font("Arial", "", 12)        pdf.ln(10)        # Adicionar conteúdo para cada destino        for destino_info in planejamento_viagem:            # Título da cidade            pdf.set_font("Arial", "B", 14)            cidade = destino_info.get('cidade', 'N/A')            pais = destino_info.get('pais', 'N/A')            pdf.cell(200, 10, txt=f"Cidade: {cidade} ({pais})", ln=True)                        # Informações básicas            pdf.set_font("Arial", "", 12)            hospedagem = destino_info.get('hospedagem', 'N/A')            pdf.multi_cell(0, 8, txt=f"Hospedagem: {hospedagem}")                        data_chegada = destino_info.get('data_chegada', 'N/A')            data_partida = destino_info.get('data_partida', 'N/A')            pdf.multi_cell(0, 8, txt=f"Período: {data_chegada} a {data_partida}")            pdf.ln(5)            # Roteiro otimizado            if "roteiro_otimizado" in destino_info and destino_info["roteiro_otimizado"]:                pdf.set_font("Arial", "B", 12)                pdf.cell(200, 10, txt="Roteiro Otimizado:", ln=True)                pdf.set_font("Arial", "", 10)                                for dia_roteiro in destino_info["roteiro_otimizado"]:                    dia = dia_roteiro.get('dia', 'N/A')                    atividades = dia_roteiro.get('atividades', 'N/A')                    pdf.multi_cell(0, 6, txt=f"Dia {dia}: {atividades}")                                        # Restaurantes                    if "restaurantes" in dia_roteiro and dia_roteiro["restaurantes"]:                        pdf.multi_cell(0, 6, txt="  Sugestões de Restaurantes:")                        for categoria, restaurantes in dia_roteiro["restaurantes"].items():                            pdf.multi_cell(0, 6, txt=f"    {categoria}:")                            for restaurante in restaurantes:                                nome = restaurante.get('nome', 'N/A')                                faixa_preco = restaurante.get('faixa_preco', 'N/A')                                pdf.multi_cell(0, 6, txt=f"      - {nome} ({faixa_preco})")                                        # Detalhes das atrações                    if "atracoes_detalhadas" in dia_roteiro and dia_roteiro["atracoes_detalhadas"]:                        pdf.multi_cell(0, 6, txt="  Detalhes das Atrações:")                        for atracao in dia_roteiro["atracoes_detalhadas"]:                            nome = atracao.get('nome', 'N/A')                            preco = atracao.get('preco_ingresso', 'N/A')                            link = atracao.get('link_ingresso', 'N/A')                            pdf.multi_cell(0, 6, txt=f"    - {nome}: Preço: {preco}, Link: {link}")            else:                pdf.multi_cell(0, 8, txt="Nenhum roteiro otimizado disponível para esta cidade.")                        pdf.ln(10)        # Salvar PDF        pdf.output(pdf_output_path)        print(f"✅ Planejamento completo da viagem salvo em PDF: {pdf_output_path}")            except Exception as e:        print(f"❌ Erro ao gerar o PDF: {e}")